## CS310 Natural Language Processing
## Lab 9: Explore BERT

In [2]:
from pprint import pprint
from typing import List
import torch
import torch.nn.functional as F

### T1. Explore Pretrained BERT Model

In this task, you will explore the pretrained BERT model using the Hugging Face Transformers library. 

First, you will load a pretrained BERT model and the correponding tokenizer. If you use the default model string `'bert-base-uncased'`, it will automatically download the model.

In our case, to avoid any network issue, you can follow these steps to load the model locally:
- Download the `bert-base-uncased.zip` file from the course website and unzip it to the folder `bert-base-uncased` in the same directory as this notebook. 
- When you load the model, you simply specify the folder path `bert-base-uncased/` (which contains all model files) to the `from_pretrained()` function. 
- *Note* that don't exclude the last `/` in the path.

In [3]:
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased/') # Make sure you download the model files first
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased/')

Some weights of the model checkpoint at bert-base-uncased/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Let's start by counting the number of parameters in the model.

In [4]:
n_tensors = 0
for param in bert_model.parameters():
    n_tensors += 1

print("Number of tensors: ", n_tensors)

Number of tensors:  199


In [5]:
n_params = 0
for param in bert_model.parameters():
    n_params += param.numel()

print("Number of parameters: ", n_params)

Number of parameters:  109482240


Next, if you are interested in how the parameters are organized, you can print the model's `_modules` attribute.

In [6]:
print(bert_model._modules)

OrderedDict([('embeddings', BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)), ('encoder', BertEncoder(
  (layer): ModuleList(
    (0-11): 12 x BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate)

In theory, you can access the parameters at any layer of the model, by specifying the layer name and index. 

For example, if you want to check the the query matrix $W^Q$ in the self-attention layer of the first transformer block, you can do the following:

In [7]:
pprint(bert_model._modules['encoder']._modules['layer'][0]._modules['attention']._modules['self']._modules['query'])

Linear(in_features=768, out_features=768, bias=True)


As you can see, the $W^Q$ matrix is implemented as a `nn.Linear` module.

Also, the same inquiry can be simplified by using the `get_submodule()` function.

In [8]:
W_q = bert_model.get_submodule('encoder.layer.0.attention.self.query')
print(W_q)
print(W_q.weight.shape)

Linear(in_features=768, out_features=768, bias=True)
torch.Size([768, 768])


### T2. Get Contextual Embeddings from BERT

Let's move on and use the BERT model to get contextual embeddings for given texts.

First, we prepare some sentences:

In [9]:
text = (
        'I have a new CPU!\n'
        'I have a new Intel CPU!\n'
        'I have a new GPU!\n'
        'I have a new NVIDIA GPU!'
    )

sentences = text.split('\n')
print(sentences)

['I have a new CPU!', 'I have a new Intel CPU!', 'I have a new GPU!', 'I have a new NVIDIA GPU!']


Try the `tokenize()` function of the previously initialized BERT tokenizer on each sentence:

In [10]:
### START YOUR CODE ###
# tokens_in_string: List[str] = None
tokens_in_string = [bert_tokenizer.tokenize(sentence) for sentence in sentences]
### END YOUR CODE ###

# Test
pprint(tokens_in_string)
# You should expect to see the following output:
# [['i', 'have', 'a', 'new', 'cpu', '!'],
#  ['i', 'have', 'a', 'new', 'intel', 'cpu', '!'],
#  ['i', 'have', 'a', 'new', 'gp', '##u', '!'],
#  ['i', 'have', 'a', 'new', 'n', '##vid', '##ia', 'gp', '##u', '!']]

[['i', 'have', 'a', 'new', 'cpu', '!'],
 ['i', 'have', 'a', 'new', 'intel', 'cpu', '!'],
 ['i', 'have', 'a', 'new', 'gp', '##u', '!'],
 ['i', 'have', 'a', 'new', 'n', '##vid', '##ia', 'gp', '##u', '!']]


**Note** that "CPU" and "Intel" are recognized as whole words, but "NVIDIA" and "GPU" are not. Thus, they appear as subwords such as "##u" "##vid" in the results

The above results are not integer token IDs yet, so now use the `batch_encode()` function, with argument `return_tensors='pt'`, to convert each sentence to integer token IDs. Here `'pt'` is for PyTorch tensors.

**Note**:
- Each token is represented as an integer in `torch.int64` data type.
- By default, the tokenizer adds special tokens `[CLS]` and `[SEP]` to the beginning and end of each sentence, which correpond to the token ID `101` and `102`, respectively.

In [11]:
import torch
### START YOUR CODE ###
token_ids_list = [torch.tensor(bert_tokenizer.encode(sentence, add_special_tokens=True)) for sentence in sentences]
### END YOUR CODE ###

# Test
print(token_ids_list[0].dtype)
print(token_ids_list)

# You should expect to see the following output:
# torch.int64
# [tensor([[  101,  1045,  2031,  1037,  2047, 17368,   999,   102]]),
#  tensor([[  101,  1045,  2031,  1037,  2047, 13420, 17368,   999,   102]]),
#  tensor([[  101,  1045,  2031,  1037,  2047, 14246,  2226,   999,   102]]),
#  tensor([[  101,  1045,  2031,  1037,  2047,  1050, 17258,  2401, 14246,  2226,
#            999,   102]])]

torch.int64
[tensor([  101,  1045,  2031,  1037,  2047, 17368,   999,   102]), tensor([  101,  1045,  2031,  1037,  2047, 13420, 17368,   999,   102]), tensor([  101,  1045,  2031,  1037,  2047, 14246,  2226,   999,   102]), tensor([  101,  1045,  2031,  1037,  2047,  1050, 17258,  2401, 14246,  2226,
          999,   102])]


So now `"CPU"` is tokenized to `17368`, `"Intel"` to `13420`, while `"GPU"` to `[14246, 2226]`, and `"NVIDIA"` to `[1050, 17258,  2401]`.

You can use the `ids_to_tokens` dictionary to map integer token IDs back to token strings, and use `decode()` function to convert a list of token IDs back to a sentence.

In [12]:
print(bert_tokenizer.ids_to_tokens[101])
print(bert_tokenizer.ids_to_tokens[102])
print(bert_tokenizer.ids_to_tokens[17368])
# print(bert_tokenizer.decode(token_ids_list[0].squeeze().tolist()))

[CLS]
[SEP]
cpu


Note that in last example above, we `squeeze` the token IDs first, becaseu the encoded IDs are of dimension $1\times N$, where $N$ is sentence length, because PyTorch uses first dimension as batch size.

It indicates that we can tokenize multiple sentences in one batch by using the `batch_encode_plus()` function, and specify the argument `padding=True` to pad all sentences to the same length.

In [13]:
encoded_sentences = bert_tokenizer.batch_encode_plus(sentences, return_tensors='pt', padding=True, return_attention_mask=False, return_token_type_ids=False)
print(encoded_sentences)

{'input_ids': tensor([[  101,  1045,  2031,  1037,  2047, 17368,   999,   102,     0,     0,
             0,     0],
        [  101,  1045,  2031,  1037,  2047, 13420, 17368,   999,   102,     0,
             0,     0],
        [  101,  1045,  2031,  1037,  2047, 14246,  2226,   999,   102,     0,
             0,     0],
        [  101,  1045,  2031,  1037,  2047,  1050, 17258,  2401, 14246,  2226,
           999,   102]])}


As you can see, the returned dictionary contains an item keyed by `'input_ids'`, which is exactly the token IDs we need. 

**Note**:
- It is a tensor of shape $B\times N$, where $B$ is the batch size (here, $B=4$) and $N$ is the maximum sentence length in the batch.
- The default padding token is `0`.


In the above example, we deliberately set `return_attention_mask=False` to show simpler results. 

If you set it to `True`, then then returned dictionary will also contain an item keyed by `'attention_mask'`, which is a tensor of shape $B\times N$ with `1` for real tokens and `0` for padding tokens. This information is useful for follow-up computations.

Try if you can get the attention mask tensor:

In [15]:
### START YOUR CODE ###
# encoded_sentences = None
# attn_mask = None
encoded_sentences = bert_tokenizer.batch_encode_plus(sentences, return_tensors='pt', padding=True, return_attention_mask=True, return_token_type_ids=False)

attn_mask = encoded_sentences['attention_mask']
### END YOUR CODE ###

# Test
print(attn_mask)
# You should expect to see the following output:
# tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
#         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
#         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
#         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


Now, let's obtain the contextual embeddings for the four target words `"CPU"`, `"Intel"`, `"NVIDIA"`, and `"GPU"` in our sentences.

First, pass the token IDs in one batch to the BERT model to get the output object, which has a `last_hidden_state` attribute that contains the contextual embeddings.

**Note**:
- You can manually specify `input_ids` and `attention_mask` as the input arguments to the model.
- Or you can directly pass the dictionary returned by `batch_encode_plus()` to the model, and use the `**` operator as most tutorials did:
  - `outputs = model(**encoded_sentences)`

In [16]:
bert_model.eval()

with torch.no_grad():
    ### START YOUR CODE ###
    # outputs = None
    outputs = bert_model(**encoded_sentences)
    ### END YOUR CODE ###


# Test
print(outputs.last_hidden_state.shape)
# You should expect to see the following output:
# torch.Size([4, 12, 768])

torch.Size([4, 12, 768])


Next, for ``"CPU"`` and ``"Intel"``, you can directly use the output vectors at the corresponding positions, because they are recognized as whole words.

Compute the average vector of `"CPU"`s in the first two sentences, and compute its cosine similarity with the vector of `"Intel"`.

*Hint*:
- Use `F.cosine_similarity()` function

In [18]:
### START YOUR CODE ###
# vec_cpu1 = None
# vec_cpu2 = None
# vec_cpu_avg = None
# vec_intel = None
# cos_cpu_intel = None
# Extract vectors for "CPU" and "Intel"
vec_cpu1 = outputs.last_hidden_state[0, 5]  # Vector for "CPU" in the first sentence
vec_cpu2 = outputs.last_hidden_state[1, 6]  # Vector for "CPU" in the second sentence
vec_cpu_avg = (vec_cpu1 + vec_cpu2) / 2  # Average vector of "CPU" in the first two sentences
vec_intel = outputs.last_hidden_state[1, 5]  # Vector for "Intel" in the second sentence

# Compute cosine similarity
cos_cpu_intel = F.cosine_similarity(vec_cpu_avg.unsqueeze(0), vec_intel.unsqueeze(0), dim=1)
### END YOUR CODE ###

# Test
print('cos_cpu_intel:', cos_cpu_intel.item())
# You should expect to see the following output:
# cos_cpu_intel: 0.7551645636558533

cos_cpu_intel: 0.7551645040512085


For `"NVIDIA"` and `"GPU"`, it's a bit trickier, as you need to use the sum of subword vectors to get the vector of the whole word.

In sentence 3, `"GPU"` is tokenized to `[14246, 2226]`, so you need to sum the vectors at these two positions.

In sentence 4, `"NVIDIA"` is tokenized to `[1050, 17258,  2401]`, so you need to sum the vectors at these three positions.

In [20]:
### START YOUR CODE ###
# vec_gpu1 = None
# vec_gpu2 = None
# vec_gpu = None
# vec_nvidia = None
# cos_gpu_nv = None

# Extract vectors for "NVIDIA" and "GPU"
vec_gpu1 = outputs.last_hidden_state[2, [5, 6]].sum(dim=0)  # Vector for "GPU" in the third sentence
vec_gpu2 = outputs.last_hidden_state[3, [8,9]].sum(dim=0)  # Vector for "NVIDIA" in the fourth sentence
vec_gpu = (vec_gpu1 + vec_gpu2) / 2  # Average vector of "NVIDIA GPU" in the third and fourth sentences
vec_nvidia = outputs.last_hidden_state[3, [5,6,7]].sum(dim=0)  # Vector for "NVIDIA" in the fourth sentence

# Compute cosine similarity
cos_gpu_nv = F.cosine_similarity(vec_gpu.unsqueeze(0), vec_nvidia.unsqueeze(0), dim=1)
### END YOUR CODE ###

# Test
print('cos_gpu_nv:', cos_gpu_nv.item())
# You should expect to see the following output:
# cos_gpu_nv: 0.7273837327957153

cos_gpu_nv: 0.7273838520050049


Now let's see if `"NVIDIA"` is closer to `"GPU"` than `"CPU"`, and vice versa for `"Intel"`.

In [21]:
### START YOUR CODE ###
# cos_cpu_nv = None
# cos_gpu_intel = None
cos_cpu_nv = F.cosine_similarity(vec_cpu_avg.unsqueeze(0), vec_nvidia.unsqueeze(0), dim=1)
cos_gpu_intel = F.cosine_similarity(vec_gpu.unsqueeze(0), vec_intel.unsqueeze(0), dim=1)
### END YOUR CODE ###

# Test
print('cos_cpu_nv:', cos_cpu_nv.item())
print('cos_gpu_intel:', cos_gpu_intel.item())
# You should expect to see the following output:
# cos_cpu_nv: 0.5931224226951599
# cos_gpu_intel: 0.5778647661209106

cos_cpu_nv: 0.5931224226951599
cos_gpu_intel: 0.5778650641441345


That's interesting, right?

How about the distance between the two products `"CPU"` and `"GPU"`? or between the two companies `"Intel"` and `"NVIDIA"`? Check it out yourself.

In [22]:
### START YOUR CODE ###
# cos_cpu_gpu = None
# cos_intel_nv = None
cos_cpu_gpu = F.cosine_similarity(vec_cpu_avg.unsqueeze(0), vec_gpu.unsqueeze(0), dim=1)
cos_intel_nv = F.cosine_similarity(vec_intel.unsqueeze(0), vec_nvidia.unsqueeze(0), dim=1)
### END YOUR CODE ###

# Test
print('cos_cpu_gpu:', cos_cpu_gpu.item())
print('cos_intel_nv:', cos_intel_nv.item())
# You should expect to see the following output:
# cos_cpu_gpu: 0.6914964914321899
# cos_intel_nv: 0.6179742813110352

cos_cpu_gpu: 0.6914964318275452
cos_intel_nv: 0.617974579334259


### T3. Access all hidden states

Let's be more adventurous and access all hidden states returned by the BERT model.

*Hint*: Simply set the argument `output_hidden_states=True` when calling the model.

In [23]:
bert_model.eval()

with torch.no_grad():
    ### START YOUR CODE ###
    # outputs = None
    outputs = bert_model(encoded_sentences['input_ids'], attention_mask=encoded_sentences['attention_mask'], output_hidden_states=True)
    ### END YOUR CODE ###

# Test
print(type(outputs.hidden_states))
print(len(outputs.hidden_states))
print(outputs.hidden_states[-1].shape)
print(outputs.hidden_states[-2].shape)
print(outputs.hidden_states[-3].shape)
print(outputs.hidden_states[-4].shape)

# You should expect to see the following output:
# <class 'tuple'>
# 13
# torch.Size([4, 12, 768])
# torch.Size([4, 12, 768])
# torch.Size([4, 12, 768])
# torch.Size([4, 12, 768])

<class 'tuple'>
13
torch.Size([4, 12, 768])
torch.Size([4, 12, 768])
torch.Size([4, 12, 768])
torch.Size([4, 12, 768])


Compute the average vector of the word `"CPU"` in the first sentence, using the hidden states of the last **four** layer.

In [24]:
### START YOUR CODE ###
# Extract vectors for "CPU" in the first sentence from the last four hidden states
vec_cpu_last4 = torch.stack(outputs.hidden_states[-4:])[:, 0, 5, :]

# Compute average vector
vec_cpu_avg_last4 = vec_cpu_last4.mean(dim=0)
### END YOUR CODE ###

# Test
cos = F.cosine_similarity(vec_cpu_avg_last4, vec_cpu_avg, dim=0)
print('cos:', cos.item())

# You should expect to see the following output:
# cos: 0.9002149701118469

cos: 0.9002150893211365
